In [ ]:
import tensorflow as tf
from keras.layers.core import Dense, Activation, Dropout
from keras.layers.recurrent import LSTM
from keras.models import Sequential
import time
from sklearn.preprocessing import MinMaxScaler

In [ ]:
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

Found GPU at: /device:GPU:0


In [ ]:
!nvidia-smi

Sun Dec 27 09:14:55 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.27.04    Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   36C    P0    33W / 250W |    359MiB / 16280MiB |      0%      Default |
|                               |                      |                 ERR! |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
Sentiment_dataset = pd.read_csv('Sentiment Data.csv')

In [ ]:
Sentiment_dataset

,Date,Subjectivity,Polarity,Compound,Negative,Neutral,Positive
0,6/21/2017,0.025000,0.050000,-0.313600,0.234250,0.765750,0.00000
1,6/21/2017,0.025000,0.050000,-0.313600,0.234250,0.765750,0.00000
2,6/21/2017,0.025000,0.050000,-0.313600,0.234250,0.765750,0.00000
3,6/21/2017,0.025000,0.050000,-0.313600,0.234250,0.765750,0.00000
4,6/21/2017,0.025000,0.050000,-0.313600,0.234250,0.765750,0.00000
...,...,...,...,...,...,...,...
113149,8/20/2018,0.083896,-0.002403,-0.081443,0.112643,0.839571,0.04775
113150,8/20/2018,0.083896,-0.002403,-0.081443,0.112643,0.839571,0.04775
113151,8/20/2018,0.083896,-0.002403,-0.081443,0.112643,0.839571,0.04775
113152,8/20/2018,0.083896,-0.002403,-0.081443,0.112643,0.839571,0.04775


In [ ]:
dataset = pd.read_csv('AAPL Mega.csv')

In [ ]:
dataset

,Local time,Open,High,Low,Close,Volume
0,21.06.2017 00:00:00.000 GMT+0400,144.998,145.007,144.998,145.007,0.0047
1,21.06.2017 17:30:00.000 GMT+0400,145.480,145.680,145.160,145.403,0.5300
2,21.06.2017 17:31:00.000 GMT+0400,145.404,145.650,145.404,145.510,0.4400
3,21.06.2017 17:32:00.000 GMT+0400,145.512,145.662,145.492,145.610,0.5300
4,21.06.2017 17:33:00.000 GMT+0400,145.610,145.660,145.221,145.230,0.5800
...,...,...,...,...,...,...
302271,27.07.2020 23:55:00.000 GMT+0400,379.098,379.258,378.908,379.008,6.3000
302272,27.07.2020 23:56:00.000 GMT+0400,379.048,379.238,379.007,379.208,4.1700
302273,27.07.2020 23:57:00.000 GMT+0400,379.208,379.218,378.918,379.177,4.6050
302274,27.07.2020 23:58:00.000 GMT+0400,379.197,379.347,379.067,379.188,5.2600


In [ ]:
dataset_train = pd.concat([dataset, Sentiment_dataset], axis=1)

In [ ]:
dataset_train

,Local time,Open,High,Low,Close,Volume,Date,Subjectivity,Polarity,Compound,Negative,Neutral,Positive
0,21.06.2017 00:00:00.000 GMT+0400,144.998,145.007,144.998,145.007,0.0047,6/21/2017,0.025,0.05,-0.3136,0.23425,0.76575,0.0
1,21.06.2017 17:30:00.000 GMT+0400,145.480,145.680,145.160,145.403,0.5300,6/21/2017,0.025,0.05,-0.3136,0.23425,0.76575,0.0
2,21.06.2017 17:31:00.000 GMT+0400,145.404,145.650,145.404,145.510,0.4400,6/21/2017,0.025,0.05,-0.3136,0.23425,0.76575,0.0
3,21.06.2017 17:32:00.000 GMT+0400,145.512,145.662,145.492,145.610,0.5300,6/21/2017,0.025,0.05,-0.3136,0.23425,0.76575,0.0
4,21.06.2017 17:33:00.000 GMT+0400,145.610,145.660,145.221,145.230,0.5800,6/21/2017,0.025,0.05,-0.3136,0.23425,0.76575,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
302271,27.07.2020 23:55:00.000 GMT+0400,379.098,379.258,378.908,379.008,6.3000,NaN,NaN,NaN,NaN,NaN,NaN,NaN
302272,27.07.2020 23:56:00.000 GMT+0400,379.048,379.238,379.007,379.208,4.1700,NaN,NaN,NaN,NaN,NaN,NaN,NaN
302273,27.07.2020 23:57:00.000 GMT+0400,379.208,379.218,378.918,379.177,4.6050,NaN,NaN,NaN,NaN,NaN,NaN,NaN
302274,27.07.2020 23:58:00.000 GMT+0400,379.197,379.347,379.067,379.188,5.2600,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
dataset_train = dataset_train.drop(['Local time', 'Date'], 1)

In [ ]:
dataset_train = dataset_train.dropna()

In [ ]:
dataset_train

,Open,High,Low,Close,Volume,Subjectivity,Polarity,Compound,Negative,Neutral,Positive
0,144.998,145.007,144.998,145.007,0.0047,0.025000,0.050000,-0.313600,0.234250,0.765750,0.00000
1,145.480,145.680,145.160,145.403,0.5300,0.025000,0.050000,-0.313600,0.234250,0.765750,0.00000
2,145.404,145.650,145.404,145.510,0.4400,0.025000,0.050000,-0.313600,0.234250,0.765750,0.00000
3,145.512,145.662,145.492,145.610,0.5300,0.025000,0.050000,-0.313600,0.234250,0.765750,0.00000
4,145.610,145.660,145.221,145.230,0.5800,0.025000,0.050000,-0.313600,0.234250,0.765750,0.00000
...,...,...,...,...,...,...,...,...,...,...,...
113148,213.088,213.138,213.078,213.087,0.3572,0.083896,-0.002403,-0.081443,0.112643,0.839571,0.04775
113149,213.088,213.168,213.068,213.167,0.2867,0.083896,-0.002403,-0.081443,0.112643,0.839571,0.04775
113150,213.168,213.177,213.038,213.048,0.3704,0.083896,-0.002403,-0.081443,0.112643,0.839571,0.04775
113151,213.047,213.088,213.007,213.008,0.3381,0.083896,-0.002403,-0.081443,0.112643,0.839571,0.04775


In [ ]:
dataset_train = dataset_train.reset_index().drop(['index'],1)

In [ ]:
training_set = dataset_train.values # to get the open prices only 

In [ ]:
training_set

array([[1.44998000e+02, 1.45007000e+02, 1.44998000e+02, ...,
        2.34250000e-01, 7.65750000e-01, 0.00000000e+00],
       [1.45480000e+02, 1.45680000e+02, 1.45160000e+02, ...,
        2.34250000e-01, 7.65750000e-01, 0.00000000e+00],
       [1.45404000e+02, 1.45650000e+02, 1.45404000e+02, ...,
        2.34250000e-01, 7.65750000e-01, 0.00000000e+00],
       ...,
       [2.13168000e+02, 2.13177000e+02, 2.13038000e+02, ...,
        1.12642857e-01, 8.39571429e-01, 4.77500000e-02],
       [2.13047000e+02, 2.13088000e+02, 2.13007000e+02, ...,
        1.12642857e-01, 8.39571429e-01, 4.77500000e-02],
       [2.13008000e+02, 2.13008000e+02, 2.12928000e+02, ...,
        1.12642857e-01, 8.39571429e-01, 4.77500000e-02]])

In [ ]:
X_train = []
y_train = []
for i in range(200, len(training_set)):
  X_train.append(training_set[i-200:i,:])
  y_train.append(training_set[i, 0])
X_train, y_train = np.array(X_train), np.array(y_train)

In [ ]:
  X_train.shape

(111890, 200, 11)

In [ ]:
  y_train.shape

(111890,)

In [ ]:
shape0 = X_train.shape[0]
shape1 = X_train.shape[1]
X_train = X_train.reshape(X_train.shape[0]*X_train.shape[1], 11)
X_train.shape
y_train = y_train.reshape(-1, 1)

In [ ]:
from sklearn.preprocessing import MinMaxScaler
sc_x = MinMaxScaler(feature_range = (0,1))
sc_y = MinMaxScaler(feature_range = (0,1))
X_train = sc_x.fit_transform(X_train)
y_train = sc_y.fit_transform(y_train)

In [ ]:
X_train = np.reshape(X_train, (shape0, shape1, 11)) # here we reshaping the training set 
#so it can fit what the RNN is expecting the first element in the tuple is the number of datapoints that we have which is the number of columns
# and the second element is the number of columns or the numner of time steps which is 60 in this case
# and the last element which is the number of indicators that we have and each indicator has its own clumns and rows

In [ ]:
X_train.shape

(111890, 200, 11)

In [ ]:
y_train.shape

(111890, 1)

In [ ]:
def build_model():
    with tf.device('/gpu:0'):
      model = Sequential()

      model.add(LSTM(
          input_shape=(X_train.shape[1], 11),
          units=100,
          return_sequences=True))
      model.add(Dropout(0.2))

      model.add(LSTM(
          100,
          return_sequences=True))
      model.add(Dropout(0.2))

      model.add(LSTM(
          100,
          return_sequences=True))
      model.add(Dropout(0.2))

      model.add(LSTM(
          100,
          return_sequences=False))
      model.add(Dropout(0.2))

      model.add(Dense(
          units=1))
      model.add(Activation("linear"))

      start = time.time()
      model.compile(loss="mse", optimizer="rmsprop")
      print("> Compilation Time : ", time.time() - start)
      return model

In [ ]:
model = build_model()

> Compilation Time :  0.01663351058959961


In [ ]:
model.fit(X_train, y_train, batch_size = 32, epochs = 50)

Epoch 1/50
3497/3497 [==============================] - 133s 35ms/step - loss: 0.0050
Epoch 2/50
3497/3497 [==============================] - 124s 35ms/step - loss: 6.4039e-04
Epoch 3/50
3497/3497 [==============================] - 123s 35ms/step - loss: 4.1970e-04
Epoch 4/50
3497/3497 [==============================] - 123s 35ms/step - loss: 3.3596e-04
Epoch 5/50
3497/3497 [==============================] - 123s 35ms/step - loss: 2.9477e-04
Epoch 6/50
3497/3497 [==============================] - 124s 35ms/step - loss: 2.7165e-04
Epoch 7/50
3497/3497 [==============================] - 124s 35ms/step - loss: 2.4465e-04
Epoch 8/50
3497/3497 [==============================] - 124s 35ms/step - loss: 2.3127e-04
Epoch 9/50
3497/3497 [==============================] - 124s 35ms/step - loss: 2.1933e-04
Epoch 10/50
3497/3497 [==============================] - 124s 35ms/step - loss: 2.1379e-04
Epoch 11/50
3497/3497 [==============================] - 123s 35ms/step - loss: 2.0291e-04
Epoch 12/50


In [ ]:
TestData = pd.read_csv('AAPL Test.csv')

In [ ]:
real_close = TestData.iloc[:, 4].values

In [ ]:
dataset_train

,Open,High,Low,Close,Volume,Subjectivity,Polarity,Compound,Negative,Neutral,Positive
0,144.998,145.007,144.998,145.007,0.0047,0.025000,0.050000,-0.313600,0.234250,0.765750,0.00000
1,145.480,145.680,145.160,145.403,0.5300,0.025000,0.050000,-0.313600,0.234250,0.765750,0.00000
2,145.404,145.650,145.404,145.510,0.4400,0.025000,0.050000,-0.313600,0.234250,0.765750,0.00000
3,145.512,145.662,145.492,145.610,0.5300,0.025000,0.050000,-0.313600,0.234250,0.765750,0.00000
4,145.610,145.660,145.221,145.230,0.5800,0.025000,0.050000,-0.313600,0.234250,0.765750,0.00000
...,...,...,...,...,...,...,...,...,...,...,...
112085,213.088,213.138,213.078,213.087,0.3572,0.083896,-0.002403,-0.081443,0.112643,0.839571,0.04775
112086,213.088,213.168,213.068,213.167,0.2867,0.083896,-0.002403,-0.081443,0.112643,0.839571,0.04775
112087,213.168,213.177,213.038,213.048,0.3704,0.083896,-0.002403,-0.081443,0.112643,0.839571,0.04775
112088,213.047,213.088,213.007,213.008,0.3381,0.083896,-0.002403,-0.081443,0.112643,0.839571,0.04775


In [ ]:
dataset_total = pd.concat((dataset_train, TestData.iloc[:,1:]), axis = 0)
inputs = dataset_total[len(dataset_total) - len(TestData) - 200:].values
inputs = inputs.reshape(-1,11)
inputs = sc_x.transform(inputs)
X_test = []
for i in range(200, 590):
  X_test.append(inputs[i-200:i, :])
X_test = np.array(X_test)
X_test = np.reshape(X_test, (X_test.shape[0], X_test.shape[1], 11)) # here we reshaping the training set 
#so it can fit what the RNN is expecting the first element in the tuple is the number of datapoints that we have which is the number of columns
# and the second element is the number of columns or the numner of time steps which is 60 in this case
# and the last element which is the number of indicators that we have and each indicator has its own clumns and rows

In [ ]:
predicted_close_prices = model.predict(X_test)
predicted_close_prices = sc_y.inverse_transform(predicted_close_prices)

In [ ]:
predicted_close_prices

array([[212.12096],
       [      nan],
       [      nan],
       [      nan],
       [      nan],
       [      nan],
       [      nan],
       [      nan],
       [      nan],
       [      nan],
       [      nan],
       [      nan],
       [      nan],
       [      nan],
       [      nan],
       [      nan],
       [      nan],
       [      nan],
       [      nan],
       [      nan],
       [      nan],
       [      nan],
       [      nan],
       [      nan],
       [      nan],
       [      nan],
       [      nan],
       [      nan],
       [      nan],
       [      nan],
       [      nan],
       [      nan],
       [      nan],
       [      nan],
       [      nan],
       [      nan],
       [      nan],
       [      nan],
       [      nan],
       [      nan],
       [      nan],
       [      nan],
       [      nan],
       [      nan],
       [      nan],
       [      nan],
       [      nan],
       [      nan],
       [      nan],
       [      nan],


In [ ]:
y_train1 = []
for i in range(200, len(training_set)):
  y_train1.append(training_set[i,0])
y_train1 = np.array(y_train1)
sc_y1 = MinMaxScaler(feature_range = (0,1))
y_train1 = y_train1.reshape(-1, 1)
y_train1 = sc_y1.fit_transform(y_train1)

In [ ]:
real_open = TestData.iloc[:, 1].values

In [ ]:
model1 = build_model()

> Compilation Time :  0.006190061569213867


In [ ]:
model1.fit(X_train, y_train1, batch_size = 32, epochs = 50)

NameError: ignored

In [ ]:
predicted_open_prices = model1.predict(X_test)

In [ ]:
predicted_open_prices = sc_y1.inverse_transform(predicted_open_prices)

In [ ]:
y_pred_change = np.subtract(predicted_close_prices, predicted_open_prices)/predicted_open_prices
y_real_change = np.subtract(real_close, real_open)/real_open

In [ ]:
np.set_printoptions(10)
y_real_change = y_real_change.reshape(390,1)

In [ ]:
np.concatenate((y_pred_change, y_real_change) ,1)

In [ ]:
from sklearn.metrics import accuracy_score,confusion_matrix
print(accuracy_score(np.sign(y_pred_change),np.sign(y_real_change)))
confusion_matrix(np.sign(y_real_change),np.sign(y_pred_change)) 

In [ ]:
model_json1 = model.to_json()
with open("apple_close_LSTM_50epoch.json", "w") as json_file:
    json_file.write(model_json1)
# serialize weights to HDF5
model.save_weights("apple_close_LSTM_50epoch.h5")
print("Saved model to disk")
model_json = model1.to_json()
with open("apple_open_LSTM_50epoch.json", "w") as json_file:
    json_file.write(model_json)
# serialize weights to HDF5
model1.save_weights("apple_open_LSTM_50epoch.h5")
print("Saved model to disk")